# Contagem de pixels

In [ ]:
!pip install rasterio
!pip install shapely
!pip install pyproj

In [ ]:
import rasterio
import os
import pandas as pd
from shapely.geometry import Polygon, mapping
from rasterio.features import geometry_mask
import numpy as np
from pyproj import Transformer
from google.colab import drive
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import geopandas as gpd
from scipy.stats import linregress
from rasterio.plot import show
from rasterio.mask import mask
from glob import glob
from PIL import Image

In [ ]:
drive.mount('/content/drive')

In [ ]:
csv_paths = {
    2017: '/content/drive/My Drive/SENTINEL_DATA/sentinel2/2017/output/contagem_pixels.csv',
    2018: '/content/drive/My Drive/SENTINEL_DATA/sentinel2/2018/output/contagem_pixels.csv',
    2019: '/content/drive/My Drive/GEE_Folder_Raw_40_Perc_2019/output/contagem_pixels.csv',
    2020: '/content/drive/My Drive/GEE_Folder_Raw_40_Perc_2020/output/contagem_pixels.csv',
    2021: '/content/drive/My Drive/GEE_Folder_Raw_40_Perc_2021/output/contagem_pixels.csv',
    2022: '/content/drive/My Drive/GEE_Folder_Raw_40_Perc_2022/output/contagem_pixels.csv',
    2023: '/content/drive/My Drive/GEE_Folder_Raw_40_Perc_2023/output/contagem_pixels.csv',
    2024: '/content/drive/My Drive/GEE_Folder_Raw_40_Perc_2024/output/contagem_pixels.csv',
    2025: '/content/drive/My Drive/GEE_Folder_Raw_40_Perc_2025/output/contagem_pixels.csv'
}

output_folders = {
    2017: '/content/drive/My Drive/SENTINEL_DATA/sentinel2/2017/output/',
    2018: '/content/drive/My Drive/SENTINEL_DATA/sentinel2/2018/output/',
    2019: '/content/drive/My Drive/GEE_Folder_Raw_40_Perc_2019/output/',
    2020: '/content/drive/My Drive/GEE_Folder_Raw_40_Perc_2020/output/',
    2021: '/content/drive/My Drive/GEE_Folder_Raw_40_Perc_2021/output/',
    2022: '/content/drive/My Drive/GEE_Folder_Raw_40_Perc_2022/output/',
    2023: '/content/drive/My Drive/GEE_Folder_Raw_40_Perc_2023/output/',
    2024: '/content/drive/My Drive/GEE_Folder_Raw_40_Perc_2024/output/',
    2025: '/content/drive/My Drive/GEE_Folder_Raw_40_Perc_2025/output/',
}

In [ ]:
def count_pixels(path_tif):
    with rasterio.open(path_tif) as img:
        #print(img.crs)
        data = img.read(1)
        transform = img.transform
        crs_img = img.crs  # sistema de coordenadas do .tif

        # reprojeta de EPSG:4326 (lat/lon) para o crs do .tif
        transformer = Transformer.from_crs("EPSG:4326", crs_img, always_xy=True)

        def reproject_polygon(coords):
            return Polygon([transformer.transform(x, y) for x, y in coords])

        # cordenadas provenientes do google earth engine
        arvorezinha_coords = [
            (-54.13341114332514, -31.23703975708336),
            (-54.12070820143061, -31.23703975708336),
            (-54.12070820143061, -31.22081967345758),
            (-54.13341114332514, -31.22081967345758)
        ]

        sanga_rasa_coords = [
            (-54.124553222478525, -31.268190891664986),
            (-54.10743000012745, -31.268190891664986),
            (-54.10743000012745, -31.257992809414805),
            (-54.124553222478525, -31.257992809414805)
        ]

        # reprojeta as coordenadas dos poligonos
        arvorezinha_poly = reproject_polygon(arvorezinha_coords)
        sanga_rasa_poly = reproject_polygon(sanga_rasa_coords)

        # cria mascaras
        mask1 = geometry_mask([mapping(arvorezinha_poly)], transform=transform, invert=True, out_shape=data.shape)
        mask2 = geometry_mask([mapping(sanga_rasa_poly)], transform=transform, invert=True, out_shape=data.shape)

        # 1 na watnet conta agua e 0 na unet (data == 1)
        # 0 na watnet conta terra e 1 na unet
        arvorezinha_count = np.sum((data == 1) & mask1)
        sanga_rasa_count = np.sum((data == 1) & mask2)

        return arvorezinha_count, sanga_rasa_count

In [ ]:
def count_pixels_aux(img_path):
  result = []

  # itera arquivos .tif
  for arq in sorted(os.listdir(img_path)):
      if arq.endswith(".tif"):
          path = os.path.join(img_path, arq)
          data_str = os.path.splitext(arq)[0]  # remove ".tif" para pegar a data
          try:
              arvorezinha, sanga_rasa = count_pixels(path)
              result.append({"data": data_str, "Arvorezinha": arvorezinha, "Sanga_Rasa": sanga_rasa})
          except Exception as e:
              print(f"Erro ao processar {arq}: {e}")

  # cria o DataFrame e salva em .csv
  df = pd.DataFrame(result)

  output_path = os.path.join(img_path, "contagem_pixels.csv")
  df.to_csv(output_path, index=False)

In [ ]:
count_pixels_aux(output_folders[2017])
count_pixels_aux(output_folders[2018])

In [ ]:
count_pixels_aux(output_folders[2019])
count_pixels_aux(output_folders[2020])
count_pixels_aux(output_folders[2021])
count_pixels_aux(output_folders[2022])
count_pixels_aux(output_folders[2023])
count_pixels_aux(output_folders[2024])
count_pixels_aux(output_folders[2025])

In [ ]:
def show_areas(img_path):
    # coordenadas originais (EPSG:4326)
    arvorezinha_coords = [
            (-54.13341114332514, -31.23703975708336),
            (-54.12070820143061, -31.23703975708336),
            (-54.12070820143061, -31.22081967345758),
            (-54.13341114332514, -31.22081967345758)
    ]

    sanga_rasa_coords = [
            (-54.124553222478525, -31.268190891664986),
            (-54.10743000012745, -31.268190891664986),
            (-54.10743000012745, -31.257992809414805),
            (-54.124553222478525, -31.257992809414805)
    ]

    for arq in sorted(os.listdir(img_path)):
        if arq.endswith(".tif"):
            path = os.path.join(img_path, arq)
            data_str = os.path.splitext(arq)[0]
            try:
                with rasterio.open(path) as img:
                    data = img.read(1)
                    crs_img = img.crs
                    transform = img.transform

                    transformer = Transformer.from_crs("EPSG:4326", crs_img, always_xy=True)
                    arvorezinha_poly = Polygon([transformer.transform(x, y) for x, y in arvorezinha_coords])
                    sanga_rasa_poly = Polygon([transformer.transform(x, y) for x, y in sanga_rasa_coords])

                    arvorezinha_gdf = gpd.GeoDataFrame(geometry=[arvorezinha_poly], crs=crs_img)
                    sanga_rasa_gdf = gpd.GeoDataFrame(geometry=[sanga_rasa_poly], crs=crs_img)

                    fig, ax = plt.subplots(figsize=(10, 8))
                    show(data, transform=transform, ax=ax, cmap='gray')
                    arvorezinha_gdf.boundary.plot(ax=ax, color='blue', label='Arvorezinha')
                    sanga_rasa_gdf.boundary.plot(ax=ax, color='green', label='Sanga Rasa')
                    plt.title(f"Áreas para contagem - Data: {data_str}")
                    plt.legend()
                    plt.show()
            except Exception as e:
                print(f"Erro ao processar {arq}: {e}")

In [ ]:
show_areas(output_folders[2017])
#show_areas(output_folders[2023])

# Análise da contagem de pixels

In [ ]:
''''def plot_count_pixels(csv_file_path, year):
    df = pd.read_csv(csv_file_path)

    # converte a coluna 'data' para datetime
    df['data'] = pd.to_datetime(df['data'])
    df.replace("?", np.nan, inplace=True)

    df[['Arvorezinha', 'Sanga_Rasa']] = df[['Arvorezinha', 'Sanga_Rasa']].apply(pd.to_numeric)

    # interpolação pra grantir continuidade das linhas no grafico
    df[['Arvorezinha', 'Sanga_Rasa']] = df[['Arvorezinha', 'Sanga_Rasa']].interpolate(method='linear')

    df = df.sort_values('data')

    plt.figure(figsize=(12, 6))
    plt.plot(df['data'], df['Arvorezinha'], marker='o', linestyle='-', label='Arvorezinha', color='green')
    plt.plot(df['data'], df['Sanga_Rasa'], marker='s', linestyle='--', label='Sanga Rasa', color='blue')

    # título e eixos
    plt.title(f'Contagem de Pixels de Água ao Longo de {year}')
    plt.xlabel('Data')
    plt.ylabel('Contagem de Pixels (água)')

    # grade e legenda
    plt.grid(True)
    plt.legend()

    plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))

    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()''''

In [ ]:
def plot_count_pixels(csv_file_path, year):
    # tenta ler o arquivo, se não achar, avisa e para a execução da função
    try:
        # lê o csv, garantindo que a coluna 'data' seja lida como texto (string)
        # isso resolve a ambiguidade entre os formatos YYYYMMDD e YYYY-MM-DD
        df = pd.read_csv(csv_file_path, dtype={'data': str})
    except FileNotFoundError:
        print(f"ARQUIVO NÃO ENCONTRADO. Arquivo: {csv_file_path}")
        return

    # troca os '?' por NaN, um valor que o pandas entende como ausente
    df.replace("?", np.nan, inplace=True)

    # converte a coluna 'data' para datetime
    # o pandas se vira pra converter os diferentes formatos de data que são texto
    df['data'] = pd.to_datetime(df['data'])

    # transforma as colunas de interesse em tipo numérico
    # o 'coerce' força erros de conversão a virarem NaN, o que evita que o script quebre
    df[['Arvorezinha', 'Sanga_Rasa']] = df[['Arvorezinha', 'Sanga_Rasa']].apply(pd.to_numeric, errors='coerce')

    # interpola os valores ausentes pra linha do gráfico não ter falhas
    df[['Arvorezinha', 'Sanga_Rasa']] = df[['Arvorezinha', 'Sanga_Rasa']].interpolate(method='linear')

    # garante que as datas estejam em ordem cronológica
    df = df.sort_values('data')

    # aqui começa a criação do gráfico
    plt.figure(figsize=(12, 6))
    plt.plot(df['data'], df['Arvorezinha'], marker='o', linestyle='-', label='Arvorezinha', color='green')
    plt.plot(df['data'], df['Sanga_Rasa'], marker='s', linestyle='--', label='Sanga Rasa', color='blue')

    # título e nome dos eixos
    plt.title(f'Contagem de Pixels de Água ao Longo de {year}')
    plt.xlabel('Data')
    plt.ylabel('Contagem de Pixels (água)')

    # grade e legenda pra ficar mais fácil de ler
    plt.grid(True)
    plt.legend()

    # ajusta como as datas são mostradas no eixo x (um marcador por mês)
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))

    # gira os nomes das datas pra não ficarem sobrepostos
    plt.xticks(rotation=45)
    plt.tight_layout() # ajusta o gráfico pra tudo caber na imagem
    plt.show()


In [ ]:
plot_count_pixels(csv_paths[2017], '2017')

In [ ]:
plot_count_pixels(csv_paths[2018], '2018')

In [ ]:
plot_count_pixels(csv_paths[2019], '2019')

In [ ]:
plot_count_pixels(csv_paths[2020], '2020')

In [ ]:
plot_count_pixels(csv_paths[2021], '2021')

In [ ]:
plot_count_pixels(csv_paths[2022], '2022')

In [ ]:
plot_count_pixels(csv_paths[2023], '2023')

In [ ]:
plot_count_pixels(csv_paths[2024], '2024')

In [ ]:
plot_count_pixels(csv_paths[2025], '2025')

In [ ]:
# listas para armazenar os dados processados de cada barragem
arvorezinha_data = []
sanga_rasa_data = []

# processa cada arquivo csv listado em csv_paths
for year, path in csv_paths.items():
    # tenta ler o arquivo; se não encontrar, avisa e pula para o próximo ano
    try:
        # lê o csv, forçando a coluna 'data' a ser lida como texto (string)
        # isso ajuda o pandas a entender melhor os formatos YYYYMMDD e YYYY-MM-DD
        df = pd.read_csv(path, dtype={'data': str})
    except FileNotFoundError:
        print(f"ARQUIVO NÃO ENCONTRADO. Arquivo: {path}, pulando o ano {year}")
        continue # vai para a próxima iteração do loop

    # substitui "?" por NaN (Not a Number) para o pandas lidar melhor com dados ausentes
    df.replace("?", np.nan, inplace=True)

    # converte a coluna 'data' para o formato datetime
    # o pandas é bom em adivinhar o formato correto quando a coluna é texto
    df['data'] = pd.to_datetime(df['data']) # convertendo a coluna 'data' original

    # converte colunas de contagem de pixels para tipo numérico
    # 'errors='coerce'' faz com que qualquer valor não conversível vire NaN
    df[['Arvorezinha', 'Sanga_Rasa']] = df[['Arvorezinha', 'Sanga_Rasa']].apply(pd.to_numeric, errors='coerce')

    # interpola valores NaN (ausentes) para garantir a continuidade dos dados
    # isso ajuda a preencher "buracos" nas séries temporais de forma linear
    df[['Arvorezinha', 'Sanga_Rasa']] = df[['Arvorezinha', 'Sanga_Rasa']].interpolate(method='linear')

    # obtém o maior valor de pixel de água por barragem no ano
    max_arvorezinha = df['Arvorezinha'].max()
    max_sanga_rasa = df['Sanga_Rasa'].max()

    # converte o valor de pixel em área (m² → km²)
    # cada pixel representa 10m x 10m = 100 m². para km², dividimos por 1.000.000
    area_arvorezinha = (max_arvorezinha * 100) / 1_000_000 if pd.notna(max_arvorezinha) else np.nan
    area_sanga_rasa = (max_sanga_rasa * 100) / 1_000_000 if pd.notna(max_sanga_rasa) else np.nan

    # armazena os dados calculados para o ano atual
    arvorezinha_data.append({'year': year, 'area': area_arvorezinha})
    sanga_rasa_data.append({'year': year, 'area': area_sanga_rasa})

# cria dataframes para cada barragem a partir das listas populadas
df_arvorezinha = pd.DataFrame(arvorezinha_data)
df_sanga_rasa = pd.DataFrame(sanga_rasa_data)

# função para plotar os gráficos de área ao longo dos anos
def plot_water_area(df, title):
    plt.figure(figsize=(12, 6)) # define o tamanho da figura do gráfico

    # barras para mostrar a maior área de água registrada no ano
    plt.bar(df['year'], df['area'], color='lightblue', label='maior área do ano')
    # linha conectando os pontos anuais
    plt.plot(df['year'], df['area'], marker='o', linestyle='-', color='blue', label='valores anuais')

    # calcula a regressão linear para mostrar uma linha de tendência
    # 'df.dropna(subset=['year', 'area'])' garante que não haja NaNs na regressão
    df_cleaned = df.dropna(subset=['year', 'area'])
    if len(df_cleaned) >= 2: # precisa de pelo menos 2 pontos para uma linha
        slope, intercept, _, _, _ = linregress(df_cleaned['year'], df_cleaned['area'])
        trend_line = slope * df_cleaned['year'] + intercept
        plt.plot(df_cleaned['year'], trend_line, color='red', linestyle='--', label=f'tendência: y = {slope:.2f}x + {intercept:.2f}')
    else:
        print(f"não há dados suficientes para calcular a linha de tendência para {title}")


    # título e nomes dos eixos, com um toque de estilo
    plt.title(f'Maior área de água por ano - {title}', fontsize=14, fontweight='bold')
    plt.xlabel('Ano', fontsize=12)
    plt.ylabel('Área (km²)', fontsize=12)
    plt.xticks(df['year'].unique(), fontweight='bold') # garante que todos os anos sejam mostrados e em negrito
    plt.yticks(fontweight='bold') # valores do eixo y em negrito

    plt.grid(True) # adiciona uma grade pra facilitar a leitura
    plt.legend() # mostra a legenda do gráfico
    plt.tight_layout() # ajusta o layout pra tudo caber direitinho
    plt.show() # exibe o gráfico

# gera e exibe os gráficos para cada barragem
plot_water_area(df_arvorezinha, 'Barragem Arvorezinha')
plot_water_area(df_sanga_rasa, 'Barragem Sanga Rasa')

In [ ]:
from glob import glob

In [ ]:
# função para exibir as imagens .tif de uma pasta
def plot_tif_subplots(folder_path, year):
    # encontra todos os arquivos .tif
    tif_files = sorted(glob(os.path.join(folder_path, '*.tif')))

    if not tif_files:
        print(f"Nenhuma imagem .tif encontrada para o ano {year}")
        return

    num_images = len(tif_files)
    cols = 6
    rows = (num_images + cols - 1) // cols

    # cria a figura com subplots
    fig, axs = plt.subplots(rows, cols, figsize=(16, 5 * rows))
    axs = axs.flatten()

    for i, tif_path in enumerate(tif_files):
        with rasterio.open(tif_path) as src:
            img = src.read(1)
        axs[i].imshow(img, cmap='gray')
        axs[i].set_title(os.path.basename(tif_path).replace('.tif', ''), fontsize=9)
        axs[i].axis('off')

    # desativa eixos de subplots vazios
    for j in range(i + 1, len(axs)):
        axs[j].axis('off')

    fig.suptitle(f'Imagens da barragem em {year}', fontsize=16, fontweight='bold')
    plt.subplots_adjust(wspace=0.1, hspace=0.4, top=0.97)
    plt.show()

# executa para cada pasta
for year, folder in output_folders.items():
    plot_tif_subplots(folder, year)

In [ ]:
"""def plot_rgb_classification_subplots(base_folder, output_folder, year):
    # encontra todos os arquivos de classificação .tif no output
    classification_files = sorted(glob(os.path.join(output_folder, '*.tif')))

    if not classification_files:
        print(f"Nenhuma imagem de classificação encontrada para o ano {year}")
        return

    num_images = len(classification_files)
    cols = 2  # RGB + classificação lado a lado
    rows = num_images

    fig, axs = plt.subplots(rows, cols, figsize=(10, 4 * rows))
    if rows == 1:
        axs = np.expand_dims(axs, axis=0)  # garante que axs[i][j] funcione mesmo com 1 linha

    for i, class_path in enumerate(classification_files):
        date_str = os.path.basename(class_path).replace('.tif', '')

        # monta caminhos para as bandas
        band_paths = {
            'B2': os.path.join(base_folder, f'Sentinel2_L2A_Harmonizado_Export_{date_str}_B2.tif'),
            'B3': os.path.join(base_folder, f'Sentinel2_L2A_Harmonizado_Export_{date_str}_B3.tif'),
            'B4': os.path.join(base_folder, f'Sentinel2_L2A_Harmonizado_Export_{date_str}_B4.tif')
        }

        try:
            # carrega as bandas
            with rasterio.open(band_paths['B2']) as b2, \
                 rasterio.open(band_paths['B3']) as b3, \
                 rasterio.open(band_paths['B4']) as b4:
                rgb = np.stack([b4.read(1), b3.read(1), b2.read(1)], axis=-1)

                # normaliza para 0-1
                rgb = rgb.astype(np.float32)
                rgb_min, rgb_max = np.percentile(rgb, 2), np.percentile(rgb, 98)
                rgb = np.clip((rgb - rgb_min) / (rgb_max - rgb_min), 0, 1)

            # carrega a classificação
            with rasterio.open(class_path) as src_class:
                class_img = src_class.read(1)

            # plota RGB
            axs[i][0].imshow(rgb)
            axs[i][0].set_title(f'{date_str} - RGB', fontsize=9)
            axs[i][0].axis('off')

            # plota classificação
            axs[i][1].imshow(class_img, cmap='gray')
            axs[i][1].set_title(f'{date_str} - Classificação', fontsize=9)
            axs[i][1].axis('off')

        except Exception as e:
            print(f"Erro ao processar {date_str}: {e}")
            axs[i][0].axis('off')
            axs[i][1].axis('off')

    fig.suptitle(f'Composição RGB e Classificação - Ano {year}', fontsize=16, fontweight='bold')
    plt.subplots_adjust(wspace=0.1, hspace=0.3, top=0.94)
    plt.show()"""

In [ ]:
def plot_rgb_classification_subplots(base_folder, output_folder, year,
                                     red_band_index=3,
                                     green_band_index=2,
                                     blue_band_index=1):
    """
    plota gráficos com imagens rgb e suas classificações, lado a lado.

    lê as bandas rgb (vermelho, verde, azul) de um único arquivo .tif multibanda.

    Args:
        base_folder (str): pasta com os arquivos .tif multibanda (ex: imagens sentinel-2).
                           os arquivos devem ter nomes como 'sentinel2_{identificador_da_data}.tif' (ex: 'sentinel2_20180103.tif').
        output_folder (str): pasta com os arquivos .tif de classificação.
                             o nome do arquivo de classificação (sem .tif, ex: '20180103') deve fornecer o {identificador_da_data}
                             para encontrar o arquivo rgb correspondente.
        year (int or str): ano para o título principal do gráfico.
        red_band_index (int): índice da banda vermelha no arquivo multibanda (base 1).
                              padrão 4 (ex: banda 4 sentinel-2).
        green_band_index (int): índice da banda verde (padrão 3).
        blue_band_index (int): índice da banda azul (padrão 2).
    """
    # localiza os arquivos .tif de classificação na pasta de saída
    classification_files = sorted(glob(os.path.join(output_folder, '*.tif')))

    if not classification_files:
        print(f"nenhuma imagem de classificação encontrada para o ano {year} em '{output_folder}'")
        return

    num_images = len(classification_files)
    cols = 2  # rgb e classificação, lado a lado
    rows = num_images

    fig, axs = plt.subplots(rows, cols, figsize=(10, 5 * rows)) # ajuste na altura da linha para melhor visualização
    if rows == 1:
        axs = np.expand_dims(axs, axis=0)  # garante que axs[i][j] funcione com apenas uma linha de gráficos

    for i, class_path in enumerate(classification_files):
        # extrai o identificador do nome do arquivo de classification.
        # este identificador deve corresponder à parte da data no nome do arquivo rgb (ex: '20180103').
        date_identifier = os.path.basename(class_path).replace('.tif', '')

        # monta o caminho pro arquivo .tif multibanda rgb
        # o nome esperado para o arquivo rgb é 'sentinel2_{date_identifier}.tif'
        multiband_rgb_path = os.path.join(base_folder, f'sentinel2_{date_identifier}.tif')

        plot_title_identifier = date_identifier # para os títulos dos subplots

        try:
            if not os.path.exists(multiband_rgb_path):
                print(f"arquivo rgb multibanda não encontrado: {multiband_rgb_path} (identificador '{plot_title_identifier}')")
                axs[i][0].set_title(f'{plot_title_identifier} - RGB (Não encontrado)', fontsize=9)
                axs[i][0].axis('off')
                # tenta carregar e exibir a classificação mesmo sem a imagem rgb
                if os.path.exists(class_path):
                    with rasterio.open(class_path) as src_class:
                        class_img = src_class.read(1)
                    axs[i][1].imshow(class_img, cmap='gray')
                    axs[i][1].set_title(f'{plot_title_identifier} - Classificação', fontsize=9)
                else:
                    axs[i][1].set_title(f'{plot_title_identifier} - Classificação (Não encontrada)', fontsize=9)
                axs[i][1].axis('off')
                continue

            # lê as bandas r, g, b do arquivo multibanda
            with rasterio.open(multiband_rgb_path) as src_rgb:
                red_band = src_rgb.read(red_band_index)
                green_band = src_rgb.read(green_band_index)
                blue_band = src_rgb.read(blue_band_index)
                # empilha as bandas para criar a imagem rgb (altura, largura, 3)
                rgb_image = np.stack([red_band, green_band, blue_band], axis=-1)

            # normaliza a imagem para exibição (valores entre 0 e 1)
            rgb_display = rgb_image.astype(np.float32)

            valid_pixels = rgb_display[np.isfinite(rgb_display)] # apenas pixels válidos (finitos)

            if valid_pixels.size == 0: # se não houver pixels válidos
                rgb_display.fill(0) # a imagem resultante é preta
            else:
                min_p = np.percentile(valid_pixels, 2)
                max_p = np.percentile(valid_pixels, 98)

                if max_p <= min_p: # caso os dados sejam constantes ou a faixa de percentil seja inválida
                    if np.allclose(valid_pixels, 0): # se os valores forem próximos de zero
                        rgb_display.fill(0) # a imagem é preta
                    else: # se for constante e não zero
                        rgb_display.fill(1.0) # a imagem é branca
                                              # (alternativa: preencher com 0.5 para cinza)
                else:
                    # suprime avisos de divisão inválida durante a normalização
                    with np.errstate(invalid='ignore', divide='ignore'):
                        rgb_display = (rgb_display - min_p) / (max_p - min_p)

            rgb_display = np.clip(rgb_display, 0, 1) # aplica clip para garantir valores no intervalo [0,1]
            rgb_display[~np.isfinite(rgb_display)] = 0 # converte nans/infs restantes para preto (0)

            # carrega a imagem da classificação
            with rasterio.open(class_path) as src_class:
                class_img = src_class.read(1)

            # exibe a imagem rgb
            axs[i][0].imshow(rgb_display)
            axs[i][0].set_title(f'{plot_title_identifier} - RGB', fontsize=9)
            axs[i][0].axis('off')

            # exibe a imagem de classification
            axs[i][1].imshow(class_img, cmap='gray') # cmap 'gray' é adequado para classificação monobanda
            axs[i][1].set_title(f'{plot_title_identifier} - Classificação', fontsize=9)
            axs[i][1].axis('off')

        except FileNotFoundError: # se não achar o arquivo de classificação
            print(f"arquivo de CLASSIFICAÇÃO não encontrado: {class_path}")
            axs[i][0].set_title(f'{plot_title_identifier} - RGB (Erro)', fontsize=9)
            axs[i][0].axis('off')
            axs[i][1].set_title(f'{plot_title_identifier} - Classificação (Erro)', fontsize=9)
            axs[i][1].axis('off')
        except rasterio.errors.RasterioIOError as e: # erro específico do rasterio
            print(f"erro de rasterio i/o ao processar '{plot_title_identifier}' (RGB: {multiband_rgb_path}, Class: {class_path}): {e}")
            axs[i][0].set_title(f'{plot_title_identifier} - RGB (Erro I/O)', fontsize=9)
            axs[i][0].axis('off')
            axs[i][1].set_title(f'{plot_title_identifier} - Classificação (Erro I/O)', fontsize=9)
            axs[i][1].axis('off')
        except Exception as e: # tratamento para outros erros
            print(f"erro geral ao processar '{plot_title_identifier}': {e}")
            axs[i][0].set_title(f'{plot_title_identifier} - RGB (Erro Inesperado)', fontsize=9)
            axs[i][0].axis('off')
            axs[i][1].set_title(f'{plot_title_identifier} - Classificação (Erro Inesperado)', fontsize=9)
            axs[i][1].axis('off')

    fig.suptitle(f'Composição RGB e Classificação - Ano {year}', fontsize=16, fontweight='bold')
    fig.tight_layout(rect=[0, 0.03, 1, 0.95]) # ajusta o layout para melhor encaixe, incluindo título principal
    plt.show()

In [ ]:
plot_rgb_classification_subplots(
    base_folder='/content/drive/My Drive/SENTINEL_DATA/sentinel2/2017/',
    output_folder='/content/drive/My Drive/SENTINEL_DATA/sentinel2/2017/output/',
    year=2017
)

In [ ]:
plot_rgb_classification_subplots(
    base_folder='/content/drive/My Drive/SENTINEL_DATA/sentinel2/2018/',
    output_folder='/content/drive/My Drive/SENTINEL_DATA/sentinel2/2018/output/',
    year=2018
)

In [ ]:
plot_rgb_classification_subplots(
    base_folder='/content/drive/My Drive/GEE_Folder_Raw_40_Perc_2019',
    output_folder='/content/drive/My Drive/GEE_Folder_Raw_40_Perc_2019/output',
    year=2019
)

In [ ]:
plot_rgb_classification_subplots(
    base_folder='/content/drive/My Drive/GEE_Folder_Raw_40_Perc_2020',
    output_folder='/content/drive/My Drive/GEE_Folder_Raw_40_Perc_2020/output',
    year=2020
)

In [ ]:
plot_rgb_classification_subplots(
    base_folder='/content/drive/My Drive/GEE_Folder_Raw_40_Perc_2021',
    output_folder='/content/drive/My Drive/GEE_Folder_Raw_40_Perc_2021/output',
    year=2021
)

In [ ]:
plot_rgb_classification_subplots(
    base_folder='/content/drive/My Drive/GEE_Folder_Raw_40_Perc_2022',
    output_folder='/content/drive/My Drive/GEE_Folder_Raw_40_Perc_2022/output',
    year=2022
)

In [ ]:
plot_rgb_classification_subplots(
    base_folder='/content/drive/My Drive/GEE_Folder_Raw_40_Perc_2023',
    output_folder='/content/drive/My Drive/GEE_Folder_Raw_40_Perc_2023/output',
    year=2023
)

In [ ]:
plot_rgb_classification_subplots(
    base_folder='/content/drive/My Drive/GEE_Folder_Raw_40_Perc_2024',
    output_folder='/content/drive/My Drive/GEE_Folder_Raw_40_Perc_2024/output',
    year=2024
)

In [ ]:
plot_rgb_classification_subplots(
    base_folder='/content/drive/My Drive/GEE_Folder_Raw_40_Perc_2025',
    output_folder='/content/drive/My Drive/GEE_Folder_Raw_40_Perc_2025/output',
    year=2025
)

In [ ]:
def gif_generate(img_path):
    df = pd.read_csv(os.path.join(img_path, "contagem_pixels.csv"))
    df["data"] = pd.to_datetime(df["data"], errors="coerce")

    coords = {
        "Arvorezinha": [
            (-54.13341114332514, -31.23703975708336),
            (-54.12070820143061, -31.23703975708336),
            (-54.12070820143061, -31.22081967345758),
            (-54.13341114332514, -31.22081967345758)
        ],
        "Sanga_Rasa": [
            (-54.124553222478525, -31.268190891664986),
            (-54.10743000012745, -31.268190891664986),
            (-54.10743000012745, -31.257992809414805),
            (-54.124553222478525, -31.257992809414805)
        ]
    }

    for barragem in ["Arvorezinha", "Sanga_Rasa"]:
        imagens = []
        grouped = df.groupby([df["data"].dt.year, df["data"].dt.month])
        selecionadas = grouped.apply(lambda x: x.loc[x[barragem].idxmax()]).reset_index(drop=True)

        for _, row in selecionadas.iterrows():
            data_str = row["data"].strftime("%Y-%m-%d")
            file_path = os.path.join(img_path, f"{row['data'].strftime('%Y-%m-%d')}.tif")

            try:
                with rasterio.open(file_path) as src:
                    crs_img = src.crs
                    transformer = Transformer.from_crs("EPSG:4326", crs_img, always_xy=True)
                    poly = Polygon([transformer.transform(x, y) for x, y in coords[barragem]])
                    geojson = [mapping(poly)]

                    cropped_image, cropped_transform = mask(src, geojson, crop=True)

                    fig, ax = plt.subplots(figsize=(6, 6))
                    show(cropped_image[0], transform=cropped_transform, ax=ax, cmap="gray")
                    plt.title(f"{barragem} - {data_str}")
                    plt.axis("off")

                    temp_img_path = os.path.join(img_path, f"_temp_{barragem}_{data_str}.png")
                    plt.savefig(temp_img_path, bbox_inches="tight", pad_inches=0)
                    plt.close()
                    imagens.append(Image.open(temp_img_path))

            except Exception as e:
                print(f"Erro com {file_path}: {e}")

        if imagens:
            gif_path = os.path.join(img_path, f"{barragem}_max_por_mes.gif")
            imagens[0].save(gif_path, save_all=True, append_images=imagens[1:], duration=1000, loop=0)

        for img in imagens:
            img.close()
        for arq in os.listdir(img_path):
            if arq.startswith("_temp_") and arq.endswith(".png"):
                os.remove(os.path.join(img_path, arq))

In [ ]:
def gif_generate(img_path):
    # caminho completo para o arquivo de contagem de pixels
    csv_file = os.path.join(img_path, "contagem_pixels.csv")

    # tenta ler o csv, se não der certo, avisa e para
    try:
        # lê o csv, já falando pro pandas que a coluna 'data' é texto por enquanto
        df = pd.read_csv(csv_file, dtype={'data': str})
    except FileNotFoundError:
        print(f"ops, não achei o csv aqui: {csv_file}")
        return
    except Exception as e:
        print(f"deu ruim ao ler o csv {csv_file}: {e}")
        return

    # troca os "?" por NaN (not a number), que é como o pandas marca dados faltantes
    df.replace("?", np.nan, inplace=True)

    # converte a coluna 'data' para o tipo datetime do pandas
    # 'errors="coerce"' faz com que datas inválidas virem NaT (not a time)
    df["data"] = pd.to_datetime(df["data"], errors="coerce")

    # remove linhas onde a data não pôde ser convertida (virou NaT)
    # isso é importante pra não tentar processar datas "quebradas" como 1970-01-01 por erro
    df.dropna(subset=["data"], inplace=True)
    if df.empty:
        print(f"ih, depois de processar as datas, o arquivo {csv_file} ficou sem dados válidos.")
        return

    # converte as colunas das barragens para número, se der erro vira NaN
    for barragem_col in ["Arvorezinha", "Sanga_Rasa"]:
        if barragem_col in df.columns:
            df[barragem_col] = pd.to_numeric(df[barragem_col], errors='coerce')
        else:
            print(f"aviso: coluna {barragem_col} não encontrada no csv {csv_file}")


    # coordenadas geográficas (latitude, longitude) das áreas de interesse (bounding box)
    coords = {
        "Arvorezinha": [
            (-54.13341114332514, -31.23703975708336),
            (-54.12070820143061, -31.23703975708336),
            (-54.12070820143061, -31.22081967345758),
            (-54.13341114332514, -31.22081967345758)
        ],
        "Sanga_Rasa": [
            (-54.124553222478525, -31.268190891664986),
            (-54.10743000012745, -31.268190891664986),
            (-54.10743000012745, -31.257992809414805),
            (-54.124553222478525, -31.257992809414805)
        ]
    }

    # faz o processo para cada barragem
    for barragem in ["Arvorezinha", "Sanga_Rasa"]:
        if barragem not in df.columns: # se a coluna da barragem nem existe no csv
            print(f"pulando {barragem} pois a coluna não existe no csv.")
            continue

        imagens = [] # lista para guardar os frames do gif
        # agrupa os dados por ano e mês da coluna 'data'
        grouped = df.groupby([df["data"].dt.year, df["data"].dt.month])

        # funçãozinha pra pegar a linha com o maior valor da barragem dentro de cada grupo (mês/ano)
        def get_row_with_max_value(x):
            if x[barragem].notna().any(): # só tenta se tiver algum valor não-NaN na coluna da barragem
                return x.loc[x[barragem].idxmax()]
            return pd.Series(dtype='object') # senão, retorna uma linha vazia pra ser filtrada depois

        # aplica a função e limpa as linhas que ficaram vazias (onde não tinha máximo)
        selecionadas = grouped.apply(get_row_with_max_value).dropna(how='all').reset_index(drop=True)

        if selecionadas.empty:
            print(f"não foram encontradas imagens/dados válidos para {barragem} após agrupar.")
            continue # pula pra próxima barragem se não sobrou nada

        # para cada imagem selecionada (maior área do mês)
        for _, row in selecionadas.iterrows():
            data_obj = row["data"] # é um objeto datetime aqui
            # formato padrão para mostrar na tela e nos nomes temporários
            display_date_str = data_obj.strftime("%Y-%m-%d")

            # formato 1: YYYY-MM-DD.tif
            filename_hyphen = f"{display_date_str}.tif"
            file_path_hyphen = os.path.join(img_path, filename_hyphen)

            # formato 2: YYYYMMDD.tif
            date_str_nodash = data_obj.strftime("%Y%m%d")
            filename_nodash = f"{date_str_nodash}.tif"
            file_path_nodash = os.path.join(img_path, filename_nodash)

            actual_file_path_to_use = None # guarda o caminho do arquivo que realmente existe

            if os.path.exists(file_path_hyphen):
                actual_file_path_to_use = file_path_hyphen
            elif os.path.exists(file_path_nodash):
                actual_file_path_to_use = file_path_nodash
            else:
                # se nenhum dos formatos existir, registra o problema e pula esta imagem
                print(f"ih, não achei o arquivo .tif para data {display_date_str} (busquei por '{filename_hyphen}' e '{filename_nodash}') em {img_path}")
                continue # pula para a próxima linha/imagem da tabela 'selecionadas'

            try:
                # tenta abrir a imagem raster (.tif) usando o caminho que encontramos
                with rasterio.open(actual_file_path_to_use) as src:
                    crs_img = src.crs # pega o sistema de coordenadas da imagem
                    # prepara o transformador de coordenadas: de WGS84 (EPSG:4326) para o da imagem
                    transformer = Transformer.from_crs("EPSG:4326", crs_img, always_xy=True)
                    # transforma as coordenadas do nosso polígono para o sistema da imagem
                    poly_coords_transformed = [transformer.transform(x, y) for x, y in coords[barragem]]
                    poly = Polygon(poly_coords_transformed)
                    geojson = [mapping(poly)] # converte o polígono pra formato geojson

                    # recorta a imagem raster usando o polígono (máscara)
                    cropped_image, cropped_transform = mask(src, geojson, crop=True)

                    # prepara a figura pra plotar a imagem recortada
                    fig, ax = plt.subplots(figsize=(6, 6)) # tamanho da figura
                    # mostra a primeira banda da imagem recortada (tons de cinza)
                    show(cropped_image[0], transform=cropped_transform, ax=ax, cmap="gray")
                    plt.title(f"{barragem} - {display_date_str}") # título com nome da barragem e data (formato padrão)
                    plt.axis("off") # desliga os eixos pra ficar mais limpo

                    # salva a imagem plotada como um png temporário (usa o formato de data padrão)
                    temp_img_path = os.path.join(img_path, f"_temp_{barragem}_{display_date_str}.png")
                    plt.savefig(temp_img_path, bbox_inches="tight", pad_inches=0)
                    plt.close(fig) # fecha a figura pra não consumir memória
                    # abre o png salvo e adiciona na lista de imagens para o gif
                    imagens.append(Image.open(temp_img_path))

            except rasterio.errors.RasterioIOError:
                # esse erro acontece se o arquivo existe mas não pode ser aberto
                print(f"xi, o arquivo {actual_file_path_to_use} existe, mas não consegui abrir.")
            except Exception as e:
                print(f"deu algum pepino com o arquivo {actual_file_path_to_use}: {e}")

        if imagens:
            gif_path = os.path.join(img_path, f"{barragem}_max_por_mes.gif")
            # salva o gif
            imagens[0].save(gif_path, save_all=True, append_images=imagens[1:], duration=1000, loop=0)
            print(f"gif salvo para {barragem} em: {gif_path}")
        else:
            print(f"sem imagens pra criar o gif da barragem {barragem}.")

        # fecha os objetos de imagem abertos e apaga os pngs temporários
        for img_obj in imagens:
            img_obj.close() # fecha cada imagem da lista

        for arq_temp in os.listdir(img_path):
            # os arquivos temporários são salvos com o display_date_str (YYYY-MM-DD)
            if arq_temp.startswith(f"_temp_{barragem}_") and arq_temp.endswith(".png"):
                try:
                    os.remove(os.path.join(img_path, arq_temp))
                except Exception as e:
                    print(f"ops, não consegui apagar o arquivo temporário {arq_temp}: {e}")

In [ ]:
gif_generate(output_folders[2017])
gif_generate(output_folders[2018])

In [ ]:
gif_generate(output_folders[2019])
gif_generate(output_folders[2020])
gif_generate(output_folders[2021])
gif_generate(output_folders[2022])
gif_generate(output_folders[2023])
gif_generate(output_folders[2024])
gif_generate(output_folders[2025])

In [ ]:
from IPython.display import Image as IPyImage, display

In [ ]:
display(IPyImage(filename=os.path.join(output_folders[2017], "Arvorezinha_max_por_mes.gif")))
display(IPyImage(filename=os.path.join(output_folders[2017], "Sanga_Rasa_max_por_mes.gif")))

In [ ]:
display(IPyImage(filename=os.path.join(output_folders[2018], "Arvorezinha_max_por_mes.gif")))
display(IPyImage(filename=os.path.join(output_folders[2018], "Sanga_Rasa_max_por_mes.gif")))

In [ ]:
display(IPyImage(filename=os.path.join(output_folders[2019], "Arvorezinha_max_por_mes.gif")))
display(IPyImage(filename=os.path.join(output_folders[2019], "Sanga_Rasa_max_por_mes.gif")))

In [ ]:
display(IPyImage(filename=os.path.join(output_folders[2020], "Arvorezinha_max_por_mes.gif")))
display(IPyImage(filename=os.path.join(output_folders[2020], "Sanga_Rasa_max_por_mes.gif")))

In [ ]:
display(IPyImage(filename=os.path.join(output_folders[2021], "Arvorezinha_max_por_mes.gif")))
display(IPyImage(filename=os.path.join(output_folders[2021], "Sanga_Rasa_max_por_mes.gif")))

In [ ]:
display(IPyImage(filename=os.path.join(output_folders[2022], "Arvorezinha_max_por_mes.gif")))
display(IPyImage(filename=os.path.join(output_folders[2022], "Sanga_Rasa_max_por_mes.gif")))

In [ ]:
display(IPyImage(filename=os.path.join(output_folders[2023], "Arvorezinha_max_por_mes.gif")))
display(IPyImage(filename=os.path.join(output_folders[2023], "Sanga_Rasa_max_por_mes.gif")))

In [ ]:
display(IPyImage(filename=os.path.join(output_folders[2024], "Arvorezinha_max_por_mes.gif")))
display(IPyImage(filename=os.path.join(output_folders[2024], "Sanga_Rasa_max_por_mes.gif")))

In [ ]:
display(IPyImage(filename=os.path.join(output_folders[2025], "Arvorezinha_max_por_mes.gif")))
display(IPyImage(filename=os.path.join(output_folders[2025], "Sanga_Rasa_max_por_mes.gif")))